In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import pickle
import tarfile
import numpy as np
import scipy as sc
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

In [6]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

--2019-08-16 00:55:29--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving webproxy (webproxy)... 10.36.19.1
Connecting to webproxy (webproxy)|10.36.19.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  1.43MB/s    in 1m 40s  

2019-08-16 00:57:10 (1.62 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [7]:
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close

<bound method TarFile.close of <tarfile.TarFile object at 0x7fcd86bcdc88>>

In [10]:
with open("cifar-10-batches-py/data_batch_1", "rb") as fo:
    data_batch = pickle.load(fo, encoding="bytes")

In [11]:
image_height = 32
image_width = 32
pixels = data_batch[b"data"].reshape(len(data_batch[b"labels"]), 3, image_width, image_height)
labels = data_batch[b"labels"]

In [13]:
rgb = 3
for i in range(len(pixels)):
    for j in range(rgb):
        final = sc.ndimage.filters.median_filter(pixels[i][j], size = (3, 3))
        pixels[i][j] = final

In [14]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
for i in range(len(pixels)):
    for j in range(rgb):
        final = clahe.apply(pixels[i][j])
        pixels[i][j] = final

In [23]:
tf.reset_default_graph()

with open("cifar-10-batches-py/test_batch", "rb") as fo:
    test_set = pickle.load(fo, encoding="bytes")

test_pixels = data_batch[b"data"].reshape(len(data_batch[b"labels"]), 3, image_width, image_height)
test_labels = data_batch[b"labels"]

x_train = pixels
y_train = labels
x_test = test_pixels
y_test = test_labels

In [24]:
x_train = pixels.astype("float32")
x_test = x_test.astype("float32")

mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train - mean)/(std + 1e-7)
x_test = (x_test - mean)/(std + 1e-7)

In [26]:
num_classes = 10

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [27]:
x_train = x_train.transpose(0, 2, 3, 1)
x_test = x_test.transpose(0, 2, 3, 1)

In [28]:
model = tf.keras.models.Sequential()

In [29]:
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding="same", input_shape = x_train.shape[1:]))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

In [30]:
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same"))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.4))

In [31]:
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), padding="same"))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.5))

In [32]:
model.add(tf.keras.layers.Flatten())

In [33]:
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation("selu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())

In [34]:
model.add(tf.keras.layers.Dense(num_classes))
model.add(tf.keras.layers.Activation("softmax"))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 64)        0         
__________

In [36]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [37]:
datagen = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)
datagen.fit(x_train)

In [38]:
batch_size = 32
epochs = 100

In [39]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), steps_per_epoch = 10000 / batch_size, epochs = epochs, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = 1)

print("Tests loss: ", score[0])
print("Tests accuracy: ", score[1])

Epoch 1/100
 90/312 [=======>......................] - ETA: 1:20 - loss: 2.6523 - acc: 0.1906

KeyboardInterrupt: 